# Author : Rute Ayalew
# Date: 9/17/2023
# Collaborators: Stack Overflow, Nicole Sung, https://github.com/Suji04/ML_from_Scratch/blob/master/decision%20tree%20classification.ipynb 


# TASK 1: Baseline Performance

In [20]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import math
import sys
from numpy import array 


I initially jumped into reading the file into a dataframe which made me encounter encoding errors and delimiter errors. I resolved the encoding error through stack overflow but the delimiter matter required me to consult a classmate who advised me to use terminal commands to identify qualities of the file first, which includes the seperator used.

Result of terminal commands:
data2d.txt has 1026 lines of ASCII text with very long lines and either tab or white spaces as seperators. 

In [21]:
df_1 = pd.read_csv('data2d.txt', encoding='cp1252', sep = '\t')

In [22]:
df_1

,Name,CAS,Inhibition,CID,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,"2,4-Di-tert-pentylphenol",120-95-6,1,8455,0,2.8913,8.359616,78.0715,46.298618,0,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
1,PD 0343701,676116-04-4,1,9984487,0,2.5278,6.389773,134.2070,72.105790,0,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
2,Triethanolamine,102-71-6,0,7618,0,-1.2981,1.685064,38.8650,24.067895,0,...,149.105193,18.669941,1.866994,10.540453,7.270323,3.270130,138.0,9.0,-2.064,36.0
3,Thiacloprid,111988-49-9,0,115224,0,0.8796,0.773696,68.2355,33.081137,0,...,252.023645,32.251316,2.015707,17.393529,0.000000,11.824332,473.0,19.0,0.963,78.0
4,Ethion,563-12-2,1,3286,0,4.1335,17.085822,95.0817,52.577446,0,...,383.987616,36.008420,1.895180,29.497457,11.461525,0.000000,792.0,24.0,4.978,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,Dipentyl phthalate,131-18-0,0,8561,0,-0.7722,0.596293,79.7904,52.224618,0,...,306.183109,43.247338,1.965788,11.085673,11.085673,0.000000,1243.0,28.0,4.784,96.0
1021,Pyrimethanil,53112-28-0,0,91650,0,0.6103,0.372466,66.1144,33.088309,0,...,199.110947,30.298153,2.019877,9.184548,0.000000,9.184548,384.0,18.0,0.858,74.0
1022,Azinphos-methyl,86-50-0,0,2268,0,2.1157,4.476186,82.2893,40.737516,0,...,317.005770,37.985703,1.999248,26.552178,8.001571,9.662814,728.0,30.0,1.463,96.0
1023,2-Chlorophenol,95-57-8,0,7245,0,1.2090,1.461681,37.2775,16.875965,0,...,128.002892,15.533410,1.941676,4.978279,2.489140,0.000000,60.0,8.0,1.312,36.0


After reading data2d.txt into a dataframe, I defined a function to identify existing datatypes and the interval of columns they exist in. 

In [23]:
#the function below is sourced from chatgpt. It creates tuples of each existing
#data type and the interval of columns in which they exist. This function offers
#understanding of what data types we are working with

def get_data_type_intervals(df):
    # Dictionary to store data types and their corresponding intervals
    data_type_intervals = {}
    
    # Initialize variables to track the current data type and its start column
    current_data_type = None
    start_column = None

    # Iterate through the columns and their data types
    for col in df.columns:
        data_type = str(df[col].dtype)  # Get the data type as a string

        if data_type != current_data_type:
            # If the data type changes, update the interval for the previous data type
            if current_data_type is not None:
                data_type_intervals[current_data_type] = (start_column, prev_column)

            # Update the current data type and start column for the new data type
            current_data_type = data_type
            start_column = col

        # Keep track of the previous column
        prev_column = col

    # Update the interval for the last data type encountered
    if current_data_type is not None:
        data_type_intervals[current_data_type] = (start_column, prev_column)

    # Convert the dictionary to a list of tuples
    result = [(data_type, interval[0], interval[1]) for data_type, interval in data_type_intervals.items()]
    return result

# Phase II: Data Understanding

In [24]:
get_data_type_intervals(df_1)

[('object', 'Name', 'CAS'),
 ('int64', 'nAtomLC', 'nAtomLAC'),
 ('float64', 'MLogP', 'Zagreb')]

I confirm my assumptions based on the provided assignment details that only the Name and CAS columns contain categorical object data types. However I notice that the Inhibition column is read as integer rather than the binary data types it is contextualized as. The nAtomLac column is the only one of the remaining columns (noted by the assignment details as computationally-derived attributes) that holds integer values rather than floats. 

Next I check for missing values with a defined function

In [25]:
'''The two defined functions below use polymorphism to report information regarding the presence
of missing data items in the passed dataframe. Both illustrate a measure of quality for the given data. 

The first function, when given only the dataframe, 
reports the total number and percentages of columns containing missing data and missing data count 
out of the entire dataframe. 

The second function take an additional argument of an integer so as to report back the
columns that contain a specified count of missing data or more.

These functions were aided by ChatGPT'''

def checkMissingVals(*args):
#the first function is executed when onyl one argument is passed
    if len(args)==1:
        df = args[0]
        missing_columns = df.columns[df.isnull().any()]
        count_col = len(missing_columns)
        count_missingData = 0
        fraction_missing_col = math.trunc((count_col/(len(df.columns))) * 100)
        fraction_missing_data = 0


        #for each col with missing values, create a list of each index containing a null
        #also updates total count of missing data values
        for col in missing_columns:
            indices_with_missing_values = df[df[col].isnull()].index.tolist()
            count_missingData = count_missingData + len(indices_with_missing_values)

            #optional statement to print each column with missing values and their indices
            #print(f"Column: {col}, Indices: {indices_with_missing_values}")

        #calculate fraction of missing values out of total potential data value counts
        total = (len(df.columns))*(len(df))
        fraction_missing_data = math.trunc((count_missingData/total) * 100)

        print("Total number of columns with missing data: " , str(count_col))
        print("Fraction of columns with missing data out of all columns: ", str(fraction_missing_col), "%")
        print("Total number of missing data values: ", str(count_missingData))
        print("Fraction of missing data values out of total potential data values: ", str(fraction_missing_data), "%")
        print("\n")

#the second function is executed when 2 arguments are passed
    if len(args)==2:
        df = args[0]
        missing_count = args[1]
        #add column to list if contains null count >= given integer
        columns_with_missing_count = df.columns[df.isnull().sum() >= missing_count]
                
        #for each col, 
        for col in columns_with_missing_count:
            count_missingData = len(df[df[col].isnull()].index.tolist())
            print(col,": ",count_missingData)

In [26]:
# this specific function inquires about the number of null data items of a specific column and how many
# this serves helpful when we are concerned with particular columns more than others
def check_col_for_null(df, col_name):
    missing_columns = df.columns[df.isnull().any()]
    missing_count = 0
    if col_name in missing_columns:
        missing_count = df[col_name].isnull().sum()
        print(col_name, "has", missing_count, " missing values")
    else:
        print(col_name, "does not have missing values.")

In [27]:
checkMissingVals(df_1)

Total number of columns with missing data:  550
Fraction of columns with missing data out of all columns:  71 %
Total number of missing data values:  10164
Fraction of missing data values out of total potential data values:  1 %




I now see that although a small percentage of the total dataframe values are null, the nulls are present in an extremely significant percentage of columns. So now I will use the 2 argument method with various values to get an image of how many columns hold a certain threshold of missing values. This step is less necessary but helpful for my sense of the data. 

In [28]:
checkMissingVals(df_1, 15)

nHBd :  19
nwHBd :  19
nHBa :  19
nwHBa :  19
nHBint2 :  19
nHBint3 :  19
nHBint4 :  19
nHBint5 :  19
nHBint6 :  19
nHBint7 :  19
nHBint8 :  19
nHBint9 :  19
nHBint10 :  19
nHsOH :  19
nHdNH :  19
nHsSH :  19
nHsNH2 :  19
nHssNH :  19
nHaaNH :  19
nHsNH3p :  19
nHssNH2p :  19
nHsssNHp :  19
nHtCH :  19
nHdCH2 :  19
nHdsCH :  19
nHaaCH :  19
nHCHnX :  19
nHCsats :  19
nHCsatu :  19
nHAvin :  19
nHother :  19
nHmisc :  19
nsLi :  19
nssBe :  19
nssssBem :  19
nsBH2 :  19
nssBH :  19
nsssB :  19
nssssBm :  19
nsCH3 :  19
ndCH2 :  19
nssCH2 :  19
ntCH :  19
ndsCH :  19
naaCH :  19
nsssCH :  19
nddC :  19
ntsC :  19
ndssC :  19
naasC :  19
naaaC :  19
nssssC :  19
nsNH3p :  19
nsNH2 :  19
nssNH2p :  19
ndNH :  19
nssNH :  19
naaNH :  19
ntN :  19
nsssNHp :  19
ndsN :  19
naaN :  19
nsssN :  19
nddsN :  19
naasN :  19
nssssNp :  19
nsOH :  19
ndO :  19
nssO :  19
naaO :  19
naOm :  19
nsOm :  19
nsF :  19
nsSiH3 :  19
nssSiH2 :  19
nsssSiH :  19
nssssSi :  19
nsPH2 :  19
nssPH :  19
nsssP : 

Now a necessary check for the specific columns that are particularly important for the problem task: Name/CID, and Inhibition

In [29]:
check_col_for_null(df_1, "Inhibition")

Inhibition does not have missing values.


Thankfully those columns have no missing values so now we are just concerned with handling the missing data of the remaining columns. Before doing so, we will determine our method to measure central tendencies. This requires us to first determine if the data is skewed or symmetric, and find the frequency of outliers if any exist.

To make my code as resuable as possible...I also defined a function to return the indices of columns containing numeric values to later slice with iloc whilst calling the pandas skew function.

In [30]:
def get_numeric_column_indices(df):
    numeric_columns = df.select_dtypes(include='number').columns
    num_columns = len(df.columns)
    iloc_index = []
    
    # Get the indices of numeric columns
    indices_numeric_columns = [df.columns.get_loc(col) for col in numeric_columns]
    iloc_index = [min(indices_numeric_columns),max(indices_numeric_columns) ]
    
    
    return iloc_index

In [31]:
iloc_index = get_numeric_column_indices(df_1)
print(iloc_index)

[2, 773]


In [32]:
df_1.iloc[:, iloc_index[0]:iloc_index[1]].skew()

Inhibition     1.075778
CID            5.304041
nAcid          3.617976
ALogP         -0.472586
ALogp2         6.353805
                ...    
WTPT-4         3.425340
WTPT-5         1.304899
WPATH         13.259141
WPOL           1.144228
XLogP          0.292968
Length: 771, dtype: float64

After attempting to determine skew through iloc calls, I decide to define a method that reports the skew in an orderly fashion

In [33]:
def skew_report(df):
    iloc_index = get_numeric_column_indices(df) #calls method to get numeric column indices
    col_skew = 0 #individual skew measurement for each column
    #counters to total each type of skew
    total_skew = 0
    left_skew = 0
    right_skew = 0 
    symmetric = 0
    
    #for each column named in the series returned by the index slice, check skew and count frequency
    for col in df.iloc[:, iloc_index[0]:iloc_index[1]]:
        col_skew = df[col].skew()
        if col_skew < 0:
            left_skew = left_skew + 1
        if col_skew > 0:
            right_skew = right_skew + 1
        if col_skew == 0:
            symmetric = symmetric + 1

    total_skew = left_skew + right_skew
    
    print("Skewness report:")
    print("Number of attributes with left skew: ", left_skew)
    print("Number of attributes with right skew: ", right_skew)
    print("Number of attributes with no skew/ are symmetric: ", symmetric)
    print("\n")
    print("Ratio of total skewed attributes to symmetric attributes = ", total_skew, ":", symmetric)    
    
    if symmetric > total_skew:
        print("Therefor, the data is mostly symmetrical")
    if total_skew > symmetric:
        print("Therefor, the data is mostly skewed")

In [34]:
skew_report(df_1)

Skewness report:
Number of attributes with left skew:  67
Number of attributes with right skew:  436
Number of attributes with no skew/ are symmetric:  268


Ratio of total skewed attributes to symmetric attributes =  503 : 268
Therefor, the data is mostly skewed


# Phase III: Data Preperation

First task of Phase III is data selection. I will first use a function to detect columns that contain no unique values to then later exclude them in the new dataframe to be created. The reasoning for this is that the columns with no unique values offer no information to our prediction model. This task will also help reduce the data size to be worked with.

In [35]:
def find_nonunique_columns(df):
    nonunique_list = []
    for column in df.columns:
        if df[column].nunique() == 1:
            nonunique_list.append(column)
    return nonunique_list

In [36]:
nonunique_list = find_nonunique_columns(df_1)
print("Total count of columns with no unique values: ",len(nonunique_list))

Total count of columns with no unique values:  268


Now we know the total count of columns with no unique values is significant. Next step will be to exclude them from our first copy of the original dataframe. 

In [37]:
#method copies the original dataframe passed into a new dataframe used to drop non-unique columns found previouisly
def remove_nonunique_columns(df):
    df_unique = df
    nonunique_list = find_nonunique_columns(df)
    for col in nonunique_list:
        if col in df_unique.columns: #minor error handling just in case
            df_unique = df_unique.drop([col],axis=1)
    return df_unique

In [38]:
#call remove_nonunique_columns() to exclude useless data and then display
df_unique = remove_nonunique_columns(df_1)
df_unique

,Name,CAS,Inhibition,CID,nAcid,ALogP,ALogp2,AMR,apol,nAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,"2,4-Di-tert-pentylphenol",120-95-6,1,8455,0,2.8913,8.359616,78.0715,46.298618,43,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
1,PD 0343701,676116-04-4,1,9984487,0,2.5278,6.389773,134.2070,72.105790,61,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
2,Triethanolamine,102-71-6,0,7618,0,-1.2981,1.685064,38.8650,24.067895,25,...,149.105193,18.669941,1.866994,10.540453,7.270323,3.270130,138.0,9.0,-2.064,36.0
3,Thiacloprid,111988-49-9,0,115224,0,0.8796,0.773696,68.2355,33.081137,25,...,252.023645,32.251316,2.015707,17.393529,0.000000,11.824332,473.0,19.0,0.963,78.0
4,Ethion,563-12-2,1,3286,0,4.1335,17.085822,95.0817,52.577446,41,...,383.987616,36.008420,1.895180,29.497457,11.461525,0.000000,792.0,24.0,4.978,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,Dipentyl phthalate,131-18-0,0,8561,0,-0.7722,0.596293,79.7904,52.224618,48,...,306.183109,43.247338,1.965788,11.085673,11.085673,0.000000,1243.0,28.0,4.784,96.0
1021,Pyrimethanil,53112-28-0,0,91650,0,0.6103,0.372466,66.1144,33.088309,28,...,199.110947,30.298153,2.019877,9.184548,0.000000,9.184548,384.0,18.0,0.858,74.0
1022,Azinphos-methyl,86-50-0,0,2268,0,2.1157,4.476186,82.2893,40.737516,31,...,317.005770,37.985703,1.999248,26.552178,8.001571,9.662814,728.0,30.0,1.463,96.0
1023,2-Chlorophenol,95-57-8,0,7245,0,1.2090,1.461681,37.2775,16.875965,13,...,128.002892,15.533410,1.941676,4.978279,2.489140,0.000000,60.0,8.0,1.312,36.0


After Phase II I learned there was a significant amount of outliers based off the vastly skewed quality of the data. Because of this, I will be using the median of each attribute to fill their null values. This decision is debatable because calculating the median becomes more costly given larger data sets, and it is important to note that for larger cases, an approximation via median intervals is better practice. 

Below is a defined function to fill missing values within each attribute by their corresponding medians using the standard pandas.median function

In [39]:
def fill_with_median(df): 
    #create new dataframe to save seperately the soon to be altered data with filled nulls
    df_filled = df
    #first select only columns with numeric data
    numeric_columns = df.select_dtypes(include='number').columns
    
    #then pick out numeric columns that have nulls and fill with median
    for col in numeric_columns:
        if df[col].isnull().any():
            df_filled[col] = df[col].fillna(df[col].median())
            
    return df_filled

Before calling the fill funciton, I check the number of nulls remaining after removing non-unique columns and see that the amount of nulls has been reduced significantly!

In [40]:
checkMissingVals(df_unique)

Total number of columns with missing data:  299
Fraction of columns with missing data out of all columns:  59 %
Total number of missing data values:  5395
Fraction of missing data values out of total potential data values:  1 %




In [41]:
df_filled = fill_with_median(df_unique)
checkMissingVals(df_filled) #check to ensure new dataframe has no missing values

Total number of columns with missing data:  0
Fraction of columns with missing data out of all columns:  0 %
Total number of missing data values:  0
Fraction of missing data values out of total potential data values:  0 %




In [42]:
df_filled.head()

,Name,CAS,Inhibition,CID,nAcid,ALogP,ALogp2,AMR,apol,nAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,"2,4-Di-tert-pentylphenol",120-95-6,1,8455,0,2.8913,8.359616,78.0715,46.298618,43,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
1,PD 0343701,676116-04-4,1,9984487,0,2.5278,6.389773,134.2070,72.105790,61,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
2,Triethanolamine,102-71-6,0,7618,0,-1.2981,1.685064,38.8650,24.067895,25,...,149.105193,18.669941,1.866994,10.540453,7.270323,3.270130,138.0,9.0,-2.064,36.0
3,Thiacloprid,111988-49-9,0,115224,0,0.8796,0.773696,68.2355,33.081137,25,...,252.023645,32.251316,2.015707,17.393529,0.000000,11.824332,473.0,19.0,0.963,78.0
4,Ethion,563-12-2,1,3286,0,4.1335,17.085822,95.0817,52.577446,41,...,383.987616,36.008420,1.895180,29.497457,11.461525,0.000000,792.0,24.0,4.978,82.0


Finally, we will do feature selection via the ANOVA F-value method because it is best for numerical inputs and categorical outputs. Before this step, I decide to define a function to move the target column of a dataframe to index 0 for ease of slicing later. 

In [43]:
#adjust the dataframe to move the target column to index 0
def move_target_column(df,target_col_name):

    col_name = str(target_col_name)
    dfcopy = df.copy()
    
    
    column_to_move = dfcopy.pop(col_name)
    target_df = column_to_move.to_frame()
    df_adjusted_final = pd.concat([target_df,dfcopy], axis=1, join='outer')

    return df_adjusted_final

Here I also define a function to make a dataframe copy only containing numeric values. This is meant to support the feature selection to be done. 

In [44]:
def make_df_only_numeric(df):
    df_numeric = df.select_dtypes(include='number')
    return df_numeric

In [45]:
df_target_moved = move_target_column(df_filled, 'Inhibition')
df_target_moved.head()

,Inhibition,Name,CAS,CID,nAcid,ALogP,ALogp2,AMR,apol,nAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,1,"2,4-Di-tert-pentylphenol",120-95-6,8455,0,2.8913,8.359616,78.0715,46.298618,43,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
1,1,PD 0343701,676116-04-4,9984487,0,2.5278,6.389773,134.2070,72.105790,61,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
2,0,Triethanolamine,102-71-6,7618,0,-1.2981,1.685064,38.8650,24.067895,25,...,149.105193,18.669941,1.866994,10.540453,7.270323,3.270130,138.0,9.0,-2.064,36.0
3,0,Thiacloprid,111988-49-9,115224,0,0.8796,0.773696,68.2355,33.081137,25,...,252.023645,32.251316,2.015707,17.393529,0.000000,11.824332,473.0,19.0,0.963,78.0
4,1,Ethion,563-12-2,3286,0,4.1335,17.085822,95.0817,52.577446,41,...,383.987616,36.008420,1.895180,29.497457,11.461525,0.000000,792.0,24.0,4.978,82.0


In [46]:
#now exclude the categorical data
df_for_feature_selection = make_df_only_numeric(df_target_moved)
df_for_feature_selection.head()

,Inhibition,CID,nAcid,ALogP,ALogp2,AMR,apol,nAtom,nHeavyAtom,nH,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,1,8455,0,2.8913,8.359616,78.0715,46.298618,43,17,26,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
1,1,9984487,0,2.5278,6.389773,134.2070,72.105790,61,31,30,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
2,0,7618,0,-1.2981,1.685064,38.8650,24.067895,25,10,15,...,149.105193,18.669941,1.866994,10.540453,7.270323,3.270130,138.0,9.0,-2.064,36.0
3,0,115224,0,0.8796,0.773696,68.2355,33.081137,25,16,9,...,252.023645,32.251316,2.015707,17.393529,0.000000,11.824332,473.0,19.0,0.963,78.0
4,1,3286,0,4.1335,17.085822,95.0817,52.577446,41,19,22,...,383.987616,36.008420,1.895180,29.497457,11.461525,0.000000,792.0,24.0,4.978,82.0


In [61]:
def feature_selection(df):
    #load libraries
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import f_classif

    #get dimensions of df for slicing
    rows = len(df.index)
    columns = len(df.columns)
        
    #Identify features and target
    x = df.iloc[:,1:columns]
    y = df.iloc[:,0].values  
    
    #Select Features with Best ANOVA F-values
    fvalue_selector = SelectKBest(f_classif, k = 100)
    x_kbest = fvalue_selector.fit_transform(x,y)
    
    print("After selecting best 3 features:", x_kbest.shape) 

    filter = fvalue_selector.get_support()
    features = array(x.columns.values.tolist())

    print("Selected best 100:")
    print(features[filter])
   
    return features[filter]

In [62]:
selected_features = feature_selection(df_for_feature_selection)

After selecting best 3 features: (1025, 100)
Selected best 100:
['ALogP' 'ALogp2' 'AMR' 'apol' 'nAtom' 'nHeavyAtom' 'nC' 'nS' 'ATSm1'
 'ATSm2' 'ATSm3' 'ATSm4' 'ATSm5' 'ATSp1' 'ATSp2' 'ATSp3' 'ATSp4' 'ATSp5'
 'BCUTp-1h' 'nBonds' 'nBonds2' 'nBondsS3' 'nBondsD' 'nBondsD2' 'nBondsM'
 'C2SP2' 'SC-3' 'VC-3' 'VC-4' 'SPC-4' 'SPC-5' 'VPC-4' 'SP-0' 'SP-1' 'SP-2'
 'SP-3' 'SP-4' 'SP-5' 'SP-6' 'SP-7' 'VP-0' 'VP-1' 'VP-2' 'VP-3' 'VP-4'
 'VP-5' 'VP-6' 'VP-7' 'CrippenLogP' 'CrippenMR' 'nwHBa' 'nHdsCH' 'nHother'
 'ndsCH' 'ndssC' 'SwHBa' 'SHdsCH' 'SHother' 'SdsCH' 'SdssC' 'maxHBd'
 'maxwHBa' 'maxdsCH' 'maxdssC' 'hmin' 'LipoaffinityIndex' 'ETA_Alpha'
 'ETA_Epsilon_3' 'ETA_dEpsilon_D' 'ETA_Beta' 'ETA_Beta_s' 'ETA_Beta_ns'
 'ETA_Eta' 'ETA_EtaP' 'ETA_Eta_R' 'ETA_Eta_F' 'ETA_Eta_L' 'ETA_Eta_R_L'
 'ETA_Eta_F_L' 'ETA_Eta_B_RC' 'Kier1' 'Kier2' 'nAtomP' 'MLogP'
 'McGowan_Volume' 'MDEC-23' 'MLFER_E' 'MLFER_L' 'nRing' 'n6Ring' 'nT6Ring'
 'nRotBt' 'VABC' 'VAdjMat' 'MW' 'WTPT-1' 'WTPT-2' 'WPOL' 'XLogP' 'Zagreb']


In [143]:
selected_features_list = selected_features.tolist()
print(selected_features_list)

['ALogP', 'ALogp2', 'AMR', 'apol', 'nAtom', 'nHeavyAtom', 'nC', 'nS', 'ATSm1', 'ATSm2', 'ATSm3', 'ATSm4', 'ATSm5', 'ATSp1', 'ATSp2', 'ATSp3', 'ATSp4', 'ATSp5', 'BCUTp-1h', 'nBonds', 'nBonds2', 'nBondsS3', 'nBondsD', 'nBondsD2', 'nBondsM', 'C2SP2', 'SC-3', 'VC-3', 'VC-4', 'SPC-4', 'SPC-5', 'VPC-4', 'SP-0', 'SP-1', 'SP-2', 'SP-3', 'SP-4', 'SP-5', 'SP-6', 'SP-7', 'VP-0', 'VP-1', 'VP-2', 'VP-3', 'VP-4', 'VP-5', 'VP-6', 'VP-7', 'CrippenLogP', 'CrippenMR', 'nwHBa', 'nHdsCH', 'nHother', 'ndsCH', 'ndssC', 'SwHBa', 'SHdsCH', 'SHother', 'SdsCH', 'SdssC', 'maxHBd', 'maxwHBa', 'maxdsCH', 'maxdssC', 'hmin', 'LipoaffinityIndex', 'ETA_Alpha', 'ETA_Epsilon_3', 'ETA_dEpsilon_D', 'ETA_Beta', 'ETA_Beta_s', 'ETA_Beta_ns', 'ETA_Eta', 'ETA_EtaP', 'ETA_Eta_R', 'ETA_Eta_F', 'ETA_Eta_L', 'ETA_Eta_R_L', 'ETA_Eta_F_L', 'ETA_Eta_B_RC', 'Kier1', 'Kier2', 'nAtomP', 'MLogP', 'McGowan_Volume', 'MDEC-23', 'MLFER_E', 'MLFER_L', 'nRing', 'n6Ring', 'nT6Ring', 'nRotBt', 'VABC', 'VAdjMat', 'MW', 'WTPT-1', 'WTPT-2', 'WPOL',

In [172]:
def compose_training_data(df, features, target_attribute):
    dfcopy = df.copy()
    training_df = pd.DataFrame()
    #add target attribute to list of features so that column is included in training data
    features.append(target_attribute)
    print(dfcopy.shape)        
    #loop checks for selected feature column name in a copy of the passed 
    #dataframe then keeps adding new columns of popped columns to create training_data df
    for col in features:
        if col in dfcopy:
            column_to_move = dfcopy.pop(col)
            col_df = pd.Series(column_to_move, name = col)
            training_df = pd.concat([training_df,col_df], axis=1, join='outer')
       
    return training_df

To compose the training data, I defined a function above that takes the cleaned dataframe, list of selected features, and a string of the target attribute that the program user would have to handwrite. I will acknowledge that there would probably be a better way to automate the inclusion of the target attribute if I had stored that attribute name in a variable earlier in the program. 

In [173]:
compose_training_data(df_for_feature_selection,selected_features_list, 'Inhibition' )
training_data = compose_training_data(df_for_feature_selection,selected_features_list, 'Inhibition' )

(1025, 504)
(1025, 504)


In [174]:
training_data.shape

(1025, 101)

In [175]:
training_data

,ALogP,ALogp2,AMR,apol,nAtom,nHeavyAtom,nC,nS,ATSm1,ATSm2,...,nRotBt,VABC,VAdjMat,MW,WTPT-1,WTPT-2,WPOL,XLogP,Zagreb,Inhibition
0,2.8913,8.359616,78.0715,46.298618,43,17,16,0,17.774469,17.332092,...,11.0,262.916605,5.087463,234.198365,32.617458,1.918674,28.0,5.417,86.0,1
1,2.5278,6.389773,134.2070,72.105790,61,31,25,1,39.342143,40.610970,...,7.0,396.668403,5.954196,434.214033,64.316532,2.074727,53.0,2.437,174.0,1
2,-1.2981,1.685064,38.8650,24.067895,25,10,6,0,12.683387,10.494822,...,9.0,149.699807,4.321928,149.105193,18.669941,1.866994,9.0,-2.064,36.0,0
3,0.8796,0.773696,68.2355,33.081137,25,16,10,1,31.280624,23.620797,...,3.0,207.791768,5.000000,252.023645,32.251316,2.015707,19.0,0.963,78.0,0
4,4.1335,17.085822,95.0817,52.577446,41,19,9,4,57.909716,55.948736,...,16.0,313.071830,5.247928,383.987616,36.008420,1.895180,24.0,4.978,82.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,-0.7722,0.596293,79.7904,52.224618,48,22,18,0,25.097876,23.992552,...,14.0,318.606337,5.459432,306.183109,43.247338,1.965788,28.0,4.784,96.0,0
1021,0.6103,0.372466,66.1144,33.088309,28,15,12,0,16.079941,16.997092,...,4.0,186.766885,4.906891,199.110947,30.298153,2.019877,18.0,0.858,74.0,0
1022,2.1157,4.476186,82.2893,40.737516,31,19,10,2,40.309268,40.524966,...,7.0,248.927356,5.247928,317.005770,37.985703,1.999248,30.0,1.463,96.0,0
1023,1.2090,1.461681,37.2775,16.875965,13,8,6,0,16.487419,10.283863,...,2.0,105.167826,4.000000,128.002892,15.533410,1.941676,8.0,1.312,36.0,0


In [176]:
training_data.to_csv(os.path.join('assignment2/output',r'training_data.csv'))

# TASK 2: SATISFACTORY PERFROMANCE

A naive model should be constructed first as a base case. The naive model will assume the most common result of the target attribute. To determine whether our naive model will make a majority classification of either 1 or 0 for Inhibition, we will quickly check the mode for the Inhibition data.

In [183]:
#quickly save the target attribute as a variable for easier code reuse in this section
target_attribute = 'Inhibition'

In [184]:
training_data[target_attribute].mode()

0    0
Name: Inhibition, dtype: int64

In [185]:
naive_target_value = training_data[target_attribute].mode()

Here I determine 0 as the mode of the target attribute and thus will be the base case value of our naive model. Now onto create a function that assumes the target attribute as 0 for comparison purposes later. 

In [220]:
def create_naive_model(df, naive_target_value, target_attribute):
    df_temp = df
    target_name = str(target_attribute)
    naive_model = pd.DataFrame()
    
    #for every row in dataframe, generate a naive value appended to a list
    #this list is to be used in concatenation later
    naive_val_list = [0] * len(df.index)

    #Iterate through column of dataframe to find target column and drop
    #this is for concatenating the prior list of naive values and a 
    #slice of dataframe excluding the original inhibition values
    for col in df_temp.columns:
        if col == target_name:
            df_temp = df_temp.drop(columns=[target_name])
    

    naive_col_df = pd.Series(naive_val_list, name = target_name)
    naive_model = pd.concat([df_temp,naive_col_df], axis=1, join='outer')
    return naive_model

In [221]:
training_data.head()

,ALogP,ALogp2,AMR,apol,nAtom,nHeavyAtom,nC,nS,ATSm1,ATSm2,...,nRotBt,VABC,VAdjMat,MW,WTPT-1,WTPT-2,WPOL,XLogP,Zagreb,Inhibition
0,2.8913,8.359616,78.0715,46.298618,43,17,16,0,17.774469,17.332092,...,11.0,262.916605,5.087463,234.198365,32.617458,1.918674,28.0,5.417,86.0,1
1,2.5278,6.389773,134.2070,72.105790,61,31,25,1,39.342143,40.610970,...,7.0,396.668403,5.954196,434.214033,64.316532,2.074727,53.0,2.437,174.0,1
2,-1.2981,1.685064,38.8650,24.067895,25,10,6,0,12.683387,10.494822,...,9.0,149.699807,4.321928,149.105193,18.669941,1.866994,9.0,-2.064,36.0,0
3,0.8796,0.773696,68.2355,33.081137,25,16,10,1,31.280624,23.620797,...,3.0,207.791768,5.000000,252.023645,32.251316,2.015707,19.0,0.963,78.0,0
4,4.1335,17.085822,95.0817,52.577446,41,19,9,4,57.909716,55.948736,...,16.0,313.071830,5.247928,383.987616,36.008420,1.895180,24.0,4.978,82.0,1


In [225]:
naive_model = create_naive_model(training_data, naive_target_value, target_attribute)

In [226]:
naive_model.head()

,ALogP,ALogp2,AMR,apol,nAtom,nHeavyAtom,nC,nS,ATSm1,ATSm2,...,nRotBt,VABC,VAdjMat,MW,WTPT-1,WTPT-2,WPOL,XLogP,Zagreb,Inhibition
0,2.8913,8.359616,78.0715,46.298618,43,17,16,0,17.774469,17.332092,...,11.0,262.916605,5.087463,234.198365,32.617458,1.918674,28.0,5.417,86.0,0
1,2.5278,6.389773,134.2070,72.105790,61,31,25,1,39.342143,40.610970,...,7.0,396.668403,5.954196,434.214033,64.316532,2.074727,53.0,2.437,174.0,0
2,-1.2981,1.685064,38.8650,24.067895,25,10,6,0,12.683387,10.494822,...,9.0,149.699807,4.321928,149.105193,18.669941,1.866994,9.0,-2.064,36.0,0
3,0.8796,0.773696,68.2355,33.081137,25,16,10,1,31.280624,23.620797,...,3.0,207.791768,5.000000,252.023645,32.251316,2.015707,19.0,0.963,78.0,0
4,4.1335,17.085822,95.0817,52.577446,41,19,9,4,57.909716,55.948736,...,16.0,313.071830,5.247928,383.987616,36.008420,1.895180,24.0,4.978,82.0,0


Now that we have our naive model with assumed Inhibition values of 0, we can move on to creating our first prediciton model: Decision Tree Induction. This code is borrowed from: Suji04 on github

In [241]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [233]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''

        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")

        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


In [238]:
#Here we are training test-split on the training_data
test_data = training_data
X = test_data.iloc[:, :-1].values
Y = test_data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [239]:
#Here we are fitting the model on the training_data
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_48 <= 3.388080000000001 ? 0.06570744374787818
 left:X_43 <= 3.2127799201067755 ? 0.021168464851713337
  left:X_7 <= 2.0 ? 0.005027855153203531
    left:X_48 <= 3.059890000000002 ? 0.004039993504336592
        left:0.0
        right:0.0
    right:1.0
  right:X_98 <= 1.7260000000000004 ? 0.11408807801266857
    left:X_30 <= 2.1226827987756227 ? 0.052914178220482444
        left:1.0
        right:0.0
    right:X_72 <= 25.26208 ? 0.14172335600907027
        left:1.0
        right:0.0
 right:X_0 <= 1.3827000000000007 ? 0.06551441646883233
  left:X_28 <= 0.2041241452319315 ? 0.044892022263450904
    left:X_78 <= 4.71461 ? 0.02828895396704645
        left:0.0
        right:0.0
    right:X_13 <= 2151.389746845459 ? 0.17999999999999994
        left:1.0
        right:0.0
  right:X_18 <= 7.801261866009959 ? 0.05692579201101927
    left:0.0
    right:X_38 <= 5.3310318245204735 ? 0.03728278594588741
        left:1.0
        right:0.0


In [240]:
#Here we are checking accuracy of decision tree model on the training_data
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.7219512195121951

The baseline performance of the first model is an accuracy score of 0.721 using the sklearn.metrics accuracy_score function. I prevent overfitting by performing feature selection during Phase II. The Decision Tree Induction code also includes an adjustable stopping criteria and split criteria insdide the 'fitting the model' code block. After the initial test with a max_depth of 3 and minimum_sample_split of 3, I then define a function that combines the fitting code block and accuracy test code blocks to automate different tests with varied inputs for the stopping & splitting criteria. Thus the accuracy with regard to complexity can be compared by using the defined function below with different max_depth and accuracy_score input. 

In [267]:
def fitModel(training_data,max_depth_val, minimum_sample_split_val):    
    #Here we are training test-split on the training_data
    test_data = training_data
    X = test_data.iloc[:, :-1].values
    Y = test_data.iloc[:, -1].values.reshape(-1,1)
    from sklearn.model_selection import train_test_split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)
    
    #Here we are fitting the model on the training_data
    classifier = DecisionTreeClassifier(min_samples_split=max_depth_val, max_depth=minimum_sample_split_val)
    classifier.fit(X_train,Y_train)
    print("PRINTING TREE FOR for max_depth = ", max_depth_val, "minimum_samples_split= ", minimum_sample_split_val)
    classifier.print_tree()
        
    #Here we are checking accuracy of decision tree model on the training_data
    Y_pred = classifier.predict(X_test) 
    from sklearn.metrics import accuracy_score
    print("\n\n\n ACCURACY SCORE for max_depth = ", max_depth_val, "minimum_samples_split= ", minimum_sample_split_val)
    print(accuracy_score(Y_test, Y_pred))
    
    return decision_tree_model_pkl


In [268]:
model_1 = fitModel(training_data, 10, 2)

PRINTING TREE FOR for max_depth =  10 minimum_samples_split=  2
X_48 <= 3.388080000000001 ? 0.06570744374787818
 left:X_43 <= 3.2127799201067755 ? 0.021168464851713337
  left:X_7 <= 2.0 ? 0.005027855153203531
    left:0.0
    right:1.0
  right:X_98 <= 1.7260000000000004 ? 0.11408807801266857
    left:0.0
    right:1.0
 right:X_0 <= 1.3827000000000007 ? 0.06551441646883233
  left:X_28 <= 0.2041241452319315 ? 0.044892022263450904
    left:0.0
    right:1.0
  right:X_18 <= 7.801261866009959 ? 0.05692579201101927
    left:0.0
    right:1.0



 ACCURACY SCORE for max_depth =  10 minimum_samples_split=  2
0.7317073170731707


After checking accuracy for a max_depth of 9 and minimum_samples_split of 2, I see a 0.01 increase in the accuracy score. Given this, I will save this model in the output directory. 

In [269]:
best_model = fitModel(training_data, 10, 2)

PRINTING TREE FOR for max_depth =  10 minimum_samples_split=  2
X_48 <= 3.388080000000001 ? 0.06570744374787818
 left:X_43 <= 3.2127799201067755 ? 0.021168464851713337
  left:X_7 <= 2.0 ? 0.005027855153203531
    left:0.0
    right:1.0
  right:X_98 <= 1.7260000000000004 ? 0.11408807801266857
    left:0.0
    right:1.0
 right:X_0 <= 1.3827000000000007 ? 0.06551441646883233
  left:X_28 <= 0.2041241452319315 ? 0.044892022263450904
    left:0.0
    right:1.0
  right:X_18 <= 7.801261866009959 ? 0.05692579201101927
    left:0.0
    right:1.0



 ACCURACY SCORE for max_depth =  10 minimum_samples_split=  2
0.7317073170731707


AttributeError: 'DecisionTreeClassifier' object has no attribute 'to_csv'

In [276]:
import pickle
import shutil
#for writing decision tree into file with pickle
    
# Dump the trained decision tree classifier with Pickle
decision_tree_pkl_filename = 'decision_tree_classifier.pkl'
# Open the file to save as pkl file
decision_tree_model_pkl = open(decision_tree_pkl_filename, 'wb')
pickle.dump(classifier, decision_tree_model_pkl)
# Close the pickle instances
decision_tree_model_pkl.close()

current_path_to_pkl = os.path.abspath('decision_tree_classifier.pkl')
shutil.move(current_path_to_pkl, "assignment2/output")

Error: Destination path 'assignment2/output/decision_tree_classifier.pkl' already exists

# TASK 3

In [279]:
evaluation_data_df = pd.read_csv('evaluation_data.txt', encoding='cp1252', sep = '\t')

In [280]:
evaluation_data_df

,Name,CAS,Inhibition,CID,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,"N,N-Dimethylaniline",121-69-7,NaN,949,0,1.1744,1.379215,44.5749,22.514723,0,...,121.089149,17.528614,1.947624,3.015355,0.000000,3.015355,88.0,9.0,1.561,40.0
1,"2,4-Di-tert-pentylphenol",120-95-6,No,8455,0,2.8913,8.359616,78.0715,46.298618,0,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
2,"1,3-Benzenediamine",108-45-2,NaN,7935,0,-0.9900,0.980100,38.0554,18.094344,0,...,108.068748,15.532958,1.941620,4.987649,0.000000,4.987649,61.0,7.0,-0.168,36.0
3,PD 0343701,676116-04-4,No,9984487,0,2.5278,6.389773,134.2070,72.105790,0,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
4,2-Chlorotoluene,95-49-8,NaN,7238,0,2.2185,4.921742,39.9540,19.167551,0,...,126.023628,15.533410,1.941676,2.489140,0.000000,0.000000,60.0,8.0,1.785,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,Benzal chloride,98-87-3,NaN,7411,0,2.4074,5.795575,45.1176,20.680758,0,...,159.984656,17.528614,1.947624,4.815165,0.000000,0.000000,88.0,9.0,2.281,40.0
758,AVE2865,648917-13-9,NaN,12148754,1,0.1404,0.019712,108.3171,56.894481,0,...,455.085968,62.136634,2.004408,29.583829,9.923603,9.560807,2930.0,50.0,2.838,160.0
759,Gentian Violet,548-62-9,No,11057,0,0.6040,0.364816,129.8111,67.303790,0,...,372.243973,56.520380,2.018585,9.109915,0.000000,9.109915,2088.0,45.0,3.976,144.0
760,Simvastatin,79902-63-9,NaN,54454,0,1.0638,1.131670,111.4563,73.348134,0,...,418.271924,60.159150,2.005305,13.649115,13.649115,0.000000,2440.0,50.0,4.775,158.0


After looking at the evaluation_data.txt file, I see a significant amount of nulls in the target attribute colummn. I also note that the target attribute in this dataset is recorded with different values of strings either Yes or No or NaN rather than binary values. I will now proceed to repeat steps of Phase II to understand the data. 

# PHASE II: Understand the data

In [281]:
get_data_type_intervals(evaluation_data_df)

[('object', 'Name', 'Inhibition'),
 ('int64', 'nAtomLC', 'nAtomLAC'),
 ('float64', 'MLogP', 'Zagreb')]

I note that the data has the same intervals of datatypes as data2d.txt. Now I will check for missing values. 

In [282]:
checkMissingVals(evaluation_data_df)

Total number of columns with missing data:  546
Fraction of columns with missing data out of all columns:  70 %
Total number of missing data values:  10214
Fraction of missing data values out of total potential data values:  1 %




In [283]:
checkMissingVals(evaluation_data_df, 15)

Inhibition :  601
nHBd :  18
nwHBd :  18
nHBa :  18
nwHBa :  18
nHBint2 :  18
nHBint3 :  18
nHBint4 :  18
nHBint5 :  18
nHBint6 :  18
nHBint7 :  18
nHBint8 :  18
nHBint9 :  18
nHBint10 :  18
nHsOH :  18
nHdNH :  18
nHsSH :  18
nHsNH2 :  18
nHssNH :  18
nHaaNH :  18
nHsNH3p :  18
nHssNH2p :  18
nHsssNHp :  18
nHtCH :  18
nHdCH2 :  18
nHdsCH :  18
nHaaCH :  18
nHCHnX :  18
nHCsats :  18
nHCsatu :  18
nHAvin :  18
nHother :  18
nHmisc :  18
nsLi :  18
nssBe :  18
nssssBem :  18
nsBH2 :  18
nssBH :  18
nsssB :  18
nssssBm :  18
nsCH3 :  18
ndCH2 :  18
nssCH2 :  18
ntCH :  18
ndsCH :  18
naaCH :  18
nsssCH :  18
nddC :  18
ntsC :  18
ndssC :  18
naasC :  18
naaaC :  18
nssssC :  18
nsNH3p :  18
nsNH2 :  18
nssNH2p :  18
ndNH :  18
nssNH :  18
naaNH :  18
ntN :  18
nsssNHp :  18
ndsN :  18
naaN :  18
nsssN :  18
nddsN :  18
naasN :  18
nssssNp :  18
nsOH :  18
ndO :  18
nssO :  18
naaO :  18
naOm :  18
nsOm :  18
nsF :  18
nsSiH3 :  18
nssSiH2 :  18
nsssSiH :  18
nssssSi :  18
nsPH2 :  18
ns

As for missing values, I note that this dataset is missing values in a similar distribution to the first dataset with the exception of the target attribute missing 601 values. Having seen this, I will skip the step of checking for nulls in the target column specifically.

Now I will determine the skew using my previously defined function. Again the purpose of this is to determine whether or not to use mean as a central tendency measurement.

In [288]:
skew_report(evaluation_data_df)

Skewness report:
Number of attributes with left skew:  61
Number of attributes with right skew:  434
Number of attributes with no skew/ are symmetric:  275


Ratio of total skewed attributes to symmetric attributes =  495 : 275
Therefor, the data is mostly skewed


After running the skew_report() function, we see that the attributes of the data2d.txt file are overwhelmingly skewed to the right. This is similar to the skew report of the first dataset. Given the abundance of skewed attributes, I assume there are a lot of outliers in the data. I use this knowledge to decide on median as the prefered measurement of central tendency because it is not influenced by extreme values.

# PHASE 3: DATA PREPERATION

Data cleaning.  
Again, I decide to eliminate nonunique columns first before filling null values with the median. 

In [291]:
nonunique_list = find_nonunique_columns(evaluation_data_df)
print("Total count of columns with no unique values: ",len(nonunique_list))

Total count of columns with no unique values:  275


I note that this dataset has a few more nonunique columns than the first set. 

In [292]:
#call remove_nonunique_columns() to exclude useless data and then display
evaluation_data_unique = remove_nonunique_columns(evaluation_data_df)
evaluation_data_unique

,Name,CAS,Inhibition,CID,nAcid,ALogP,ALogp2,AMR,apol,nAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,"N,N-Dimethylaniline",121-69-7,NaN,949,0,1.1744,1.379215,44.5749,22.514723,20,...,121.089149,17.528614,1.947624,3.015355,0.000000,3.015355,88.0,9.0,1.561,40.0
1,"2,4-Di-tert-pentylphenol",120-95-6,No,8455,0,2.8913,8.359616,78.0715,46.298618,43,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
2,"1,3-Benzenediamine",108-45-2,NaN,7935,0,-0.9900,0.980100,38.0554,18.094344,16,...,108.068748,15.532958,1.941620,4.987649,0.000000,4.987649,61.0,7.0,-0.168,36.0
3,PD 0343701,676116-04-4,No,9984487,0,2.5278,6.389773,134.2070,72.105790,61,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
4,2-Chlorotoluene,95-49-8,NaN,7238,0,2.2185,4.921742,39.9540,19.167551,15,...,126.023628,15.533410,1.941676,2.489140,0.000000,0.000000,60.0,8.0,1.785,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,Benzal chloride,98-87-3,NaN,7411,0,2.4074,5.795575,45.1176,20.680758,15,...,159.984656,17.528614,1.947624,4.815165,0.000000,0.000000,88.0,9.0,2.281,40.0
758,AVE2865,648917-13-9,NaN,12148754,1,0.1404,0.019712,108.3171,56.894481,48,...,455.085968,62.136634,2.004408,29.583829,9.923603,9.560807,2930.0,50.0,2.838,160.0
759,Gentian Violet,548-62-9,No,11057,0,0.6040,0.364816,129.8111,67.303790,58,...,372.243973,56.520380,2.018585,9.109915,0.000000,9.109915,2088.0,45.0,3.976,144.0
760,Simvastatin,79902-63-9,NaN,54454,0,1.0638,1.131670,111.4563,73.348134,68,...,418.271924,60.159150,2.005305,13.649115,13.649115,0.000000,2440.0,50.0,4.775,158.0


I note that the dataframe has been reduced by 275 columns

In [293]:
checkMissingVals(evaluation_data_unique)

Total number of columns with missing data:  289
Fraction of columns with missing data out of all columns:  57 %
Total number of missing data values:  5588
Fraction of missing data values out of total potential data values:  1 %




I note the amount of nulls has decreased by 13% after removing nonunique columns. Now I will fill the remaining nulls.

In [329]:
evaluation_data_filled = fill_with_median(evaluation_data_unique)
checkMissingVals(evaluation_data_filled) #check to ensure new dataframe has no missing values

Total number of columns with missing data:  0
Fraction of columns with missing data out of all columns:  0 %
Total number of missing data values:  0
Fraction of missing data values out of total potential data values:  0 %




In [330]:
evaluation_data_filled.head()

,Name,CAS,CID,nAcid,ALogP,ALogp2,AMR,apol,nAtom,nHeavyAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,"N,N-Dimethylaniline",121-69-7,949,0,1.1744,1.379215,44.5749,22.514723,20,9,...,121.089149,17.528614,1.947624,3.015355,0.000000,3.015355,88.0,9.0,1.561,40.0
1,"2,4-Di-tert-pentylphenol",120-95-6,8455,0,2.8913,8.359616,78.0715,46.298618,43,17,...,234.198365,32.617458,1.918674,2.540347,2.540347,0.000000,488.0,28.0,5.417,86.0
2,"1,3-Benzenediamine",108-45-2,7935,0,-0.9900,0.980100,38.0554,18.094344,16,8,...,108.068748,15.532958,1.941620,4.987649,0.000000,4.987649,61.0,7.0,-0.168,36.0
3,PD 0343701,676116-04-4,9984487,0,2.5278,6.389773,134.2070,72.105790,61,31,...,434.214033,64.316532,2.074727,18.849351,2.530983,13.193887,3051.0,53.0,2.437,174.0
4,2-Chlorotoluene,95-49-8,7238,0,2.2185,4.921742,39.9540,19.167551,15,8,...,126.023628,15.533410,1.941676,2.489140,0.000000,0.000000,60.0,8.0,1.785,36.0


Now the data is clean we will move on to rearranging the target attribute column to the end of the dataframe and removing the categorical attributes in order to apply our model!

In [332]:
def make_df_only_numeric_exception(df, target):
    target_name = str(target)
    df_numeric = df
    df_numeric_final = pd.DataFrame()
    num_list = []
    
    df_target_series = df_numeric.pop(target_name)
    
    df_target_frame = pd.Series(df_target_series, name = target_name)
    df_numeric_final = pd.concat([df_numeric,df_target_frame], axis=1, join='outer')
    
    return df_numeric_final

In [333]:
#now exclude the categorical data
#note an exception is made with altered function since Inhibition in this data is categorical
evaluation_data_for_feature_selection = make_df_only_numeric_exception(evaluation_data_filled, target_attribute)
evaluation_data_for_feature_selection.head()

KeyError: 'Inhibition'

In [324]:
compose_training_data(evaluation_data_for_feature_selection,selected_features_list, target_attribute )
prepped_data = compose_training_data(evaluation_data_for_feature_selection,selected_features_list, target_attribute)

(762, 497)
(762, 497)


In [325]:
model1_with_evaluation_data = fitModel(prepped_data, 10, 2)

TypeError: '<' not supported between instances of 'str' and 'float'